<a href="https://colab.research.google.com/github/shovalkooba/shoval1/blob/master/split%20coco%20to%20classess%20and%20save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget http://images.cocodataset.org/zips/train2017.zip


In [0]:
%%capture
!unzip train2017.zip

In [0]:
!rm train2017.zip

In [0]:
!wget http://images.cocodataset.org/zips/val2017.zip


In [0]:
%%capture
!unzip val2017.zip

In [0]:
!rm val2017.zip

In [0]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip


In [0]:
%%capture
!unzip annotations_trainval2017.zip

In [0]:
!rm annotations_trainval2017.zip

In [0]:
!wget http://images.cocodataset.org/annotations/stuff_annotations_trainval2017.zip

In [0]:
%%capture
!unzip stuff_annotations_trainval2017.zip

In [0]:
!rm stuff_annotations_trainval2017.zip

In [0]:
!pip install gluoncv

In [0]:
!pip install mxnet

In [0]:
!ls

In [0]:
from gluoncv import data, utils
from matplotlib import pyplot as plt

train_dataset = data.COCODetection('.',splits=['instances_train2017'])
val_dataset = data.COCODetection('.',splits=['instances_val2017'])
print('Num of training images:', len(train_dataset))
print('Num of validation images:', len(val_dataset))

In [0]:
print(len(train_dataset))
print(len(val_dataset))

In [0]:
train_image, train_label = train_dataset[5562]
bounding_boxes = train_label[:, :4]
class_ids = train_label[:, 4:5]
print(train_label.shape)
print('Image size (height, width, RGB):', train_image.shape)
print('Num of objects:', bounding_boxes.shape[0])
print('Bounding boxes (num_boxes, x_min, y_min, x_max, y_max):\n',
      bounding_boxes)
print('Class IDs (num_boxes, ):\n', class_ids)

utils.viz.plot_bbox(train_image.asnumpy(), bounding_boxes, scores=None,
                    labels=class_ids, class_names=train_dataset.classes)
plt.show()

In [16]:
print(len(train_dataset))

117266


In [0]:
#OpenCV demo

def imShowCv(image):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  
  
  

In [0]:
import cv2
import numpy as np


train_image, train_label = val_dataset[4852]
bounding_boxes = train_label[:, :4]
class_ids = train_label[:, 4:5]
print(train_label.shape)



print(bounding_boxes)
print(class_ids)

train_image = train_image.asnumpy()
print(type(train_image))
train_image = cv2.cvtColor(train_image, cv2.COLOR_BGR2RGB)


for i in range(len(class_ids)):
  
  class_id = str(int(class_ids[i][0]))
  bbox = bounding_boxes[i]
  
  x1 = int(bbox[0])
  y1 = int(bbox[1])
  x2 = int(bbox[2])
  y2 = int(bbox[3])
  
  
  cv2.rectangle(train_image, (x1, y1), (x2, y2), (255, 0, 0), 1)
  cv2.putText(train_image,class_id, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, .6,(0, 0, 255),2)
  
  




imShowCv(train_image)


In [0]:
def yolo_format(class_index, point_1, point_2, width, height):
    # YOLO wants everything normalized
    # Order: class x_center y_center x_width y_height
    x_center = (point_1[0] + point_2[0]) / float(2.0 * width)
    y_center = (point_1[1] + point_2[1]) / float(2.0 * height)
    x_width = float(abs(point_2[0] - point_1[0])) / width
    y_height = float(abs(point_2[1] - point_1[1])) / height
    return str(class_index) + " " + str(x_center) \
       + " " + str(y_center) + " " + str(x_width) + " " + str(y_height)
  
def save_bb(txt_path, line):
    with open(txt_path, 'a') as myfile:
        myfile.write(line + "\n") # append line

In [0]:
!mkdir bbox_txt_train
!mkdir images_train
!mkdir bbox_txt_val
!mkdir images_val

In [29]:
import cv2
import os

n=0
imgFolder = 'images_train'
txtFolder = 'bbox_txt_train'

#id of your class in training of the YOLOv3 model
label_class_id = 0



for i in range(len(train_dataset)):
  
  if i % 500 == 0:
    print(i)
    print('images of interest',n)
  
  train_image, train_label = train_dataset[i]
  bounding_boxes = train_label[:, :4]
  class_ids = train_label[:, 4:5]
  
  
  
  if 14 in class_ids:   #14 is bird class in COCO
    
    name = 'bird_'+str(n)
    txtPath = os.path.join(txtFolder, name)+'.txt'
    imgPath = os.path.join(imgFolder, name) +'.jpg'
    #print(txtPath, imgPath)
    
    
    
    
    #utils.viz.plot_bbox(train_image.asnumpy(), bounding_boxes, scores=None,
                    #labels=class_ids, class_names=train_dataset.classes)
    
    
    image = train_image.asnumpy()
    height, width = image.shape[:2]
    
    #print(bounding_boxes)
    #print(class_ids)
    
    
    for j in range(len(class_ids)):
      
      if class_ids[j]==14:
        
        
        x1 = int(bounding_boxes[j][0])
        y1 = int(bounding_boxes[j][1])
        x2 = int(bounding_boxes[j][2])
        y2 = int(bounding_boxes[j][3])
        
        
        
        line = yolo_format(label_class_id,(x1,y1),(x2,y2),width, height)
        save_bb(txtPath, line)
        
        
    n+=1
    
        
        #cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (255,0,0), 2)
    
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imwrite(imgPath,image)
    
    #plt.show()
      
print(n)

0
images of interest 0
500
images of interest 12
1000
images of interest 21
1500
images of interest 35
2000
images of interest 52
2500
images of interest 65
3000
images of interest 81
3500
images of interest 94
4000
images of interest 105
4500
images of interest 117
5000
images of interest 125
5500
images of interest 141
6000
images of interest 150
6500
images of interest 166
7000
images of interest 175
7500
images of interest 189
8000
images of interest 202
8500
images of interest 210
9000
images of interest 231
9500
images of interest 244
10000
images of interest 252
10500
images of interest 262
11000
images of interest 270
11500
images of interest 285
12000
images of interest 299
12500
images of interest 317
13000
images of interest 339
13500
images of interest 350
14000
images of interest 370
14500
images of interest 383
15000
images of interest 396
15500
images of interest 411
16000
images of interest 420
16500
images of interest 431
17000
images of interest 442
17500
images of in

In [30]:
n=0
imgFolder = 'images_val'
txtFolder = 'bbox_txt_val'

#id of your class in validation of the YOLOv3 model
label_class_id = 0



for i in range(len(val_dataset)):
  
  if i % 500 == 0:
    print(i)
    print('images of interest',n)
  
  val_image, val_label =val_dataset[i]
  bounding_boxes = val_label[:, :4]
  class_ids = val_label[:, 4:5]
  
  
  
  if 14 in class_ids:
    
    name = 'bird_'+str(n)
    txtPath = os.path.join(txtFolder, name)+'.txt'
    imgPath = os.path.join(imgFolder, name) +'.jpg'
    #print(txtPath, imgPath)
    
    
    
    
    #utils.viz.plot_bbox(train_image.asnumpy(), bounding_boxes, scores=None,
                    #labels=class_ids, class_names=train_dataset.classes)
    
    
    image = val_image.asnumpy()
    height, width = image.shape[:2]
    
    #print(bounding_boxes)
    #print(class_ids)
    
    
    for j in range(len(class_ids)):
      
      if class_ids[j]==14:
        
        
        x1 = int(bounding_boxes[j][0])
        y1 = int(bounding_boxes[j][1])
        x2 = int(bounding_boxes[j][2])
        y2 = int(bounding_boxes[j][3])
        
        
        
        line = yolo_format(label_class_id,(x1,y1),(x2,y2),width, height)
        save_bb(txtPath, line)
        
        
    n+=1
    
        
        #cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (255,0,0), 2)
    
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imwrite(imgPath,image)
    
    #plt.show()
      
print(n)

0
images of interest 0
500
images of interest 14
1000
images of interest 27
1500
images of interest 41
2000
images of interest 54
2500
images of interest 64
3000
images of interest 75
3500
images of interest 86
4000
images of interest 99
4500
images of interest 116
125


In [31]:
!zip -r label.zip bbox_txt_train images_train bbox_txt_val images_val

Streaming output truncated to the last 5000 lines.
  adding: bbox_txt_train/bird_2905.txt (deflated 54%)
  adding: bbox_txt_train/bird_3012.txt (deflated 62%)
  adding: bbox_txt_train/bird_367.txt (deflated 43%)
  adding: bbox_txt_train/bird_1027.txt (deflated 66%)
  adding: bbox_txt_train/bird_400.txt (deflated 39%)
  adding: bbox_txt_train/bird_2261.txt (deflated 44%)
  adding: bbox_txt_train/bird_2753.txt (deflated 17%)
  adding: bbox_txt_train/bird_1031.txt (deflated 18%)
  adding: bbox_txt_train/bird_1717.txt (deflated 62%)
  adding: bbox_txt_train/bird_1869.txt (deflated 12%)
  adding: bbox_txt_train/bird_616.txt (deflated 25%)
  adding: bbox_txt_train/bird_1921.txt (deflated 61%)
  adding: bbox_txt_train/bird_1622.txt (deflated 64%)
  adding: bbox_txt_train/bird_1780.txt (deflated 31%)
  adding: bbox_txt_train/bird_462.txt (deflated 14%)
  adding: bbox_txt_train/bird_1690.txt (deflated 20%)
  adding: bbox_txt_train/bird_1615.txt (deflated 42%)
  adding: bbox_txt_train/bird_1392.

In [32]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [33]:
print(len(os.listdir('images_train')))
print(len(os.listdir('bbox_txt_train')))
print(len(os.listdir('images_val')))
print(len(os.listdir('bbox_txt_val')))

3236
3236
125
125
